# MyUtils.jl

* Copyright (c) 2021 Gen Kuroki
* License: https://opensource.org/licenses/MIT
* Repository: https://github.com/genkuroki/MyUtils.jl
* nbviewer: https://nbviewer.jupyter.org/github/genkuroki/MyUtils.jl/blob/main/MyUtils.ipynb

__Table of Contents__

<div class="toc"><ul class="toc-item"><li><span><a href="#@my_threads-and-@my_distributed" data-toc-modified-id="@my_threads-and-@my_distributed-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><code>@my_threads</code> and <code>@my_distributed</code></a></span></li><li><span><a href="#printf-functions" data-toc-modified-id="printf-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>printf functions</a></span></li><li><span><a href="#RemoveOneFrom(A::AbstractVector,-k::Integer)-&lt;:-AbstractVector" data-toc-modified-id="RemoveOneFrom(A::AbstractVector,-k::Integer)-<:-AbstractVector-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><code>RemoveOneFrom(A::AbstractVector, k::Integer) &lt;: AbstractVector</code></a></span></li><li><span><a href="#@savevar-and-@loadvar" data-toc-modified-id="@savevar-and-@loadvar-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><code>@savevar</code> and <code>@loadvar</code></a></span></li><li><span><a href="#@defunpack" data-toc-modified-id="@defunpack-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><code>@defunpack</code></a></span></li></ul></div>

In [1]:
@show VERSION
isfile("Project.toml") && using Revise
using MyUtils

VERSION = v"1.7.0-beta3.0"


┌ Info: Precompiling MyUtils [a905a9d9-1f33-49e1-8b11-bf004baf7ee1]
└ @ Base loading.jl:1423


## `@my_threads` and `@my_distributed`

In [2]:
using Base.Threads
using Distributed
using Random

@show nthreads();

nthreads() = 12


In [3]:
?MyUtils.@my_threads

```
@my_threads
```

A macro to parallelize a `for` loop to run with multiple threads.  It splits the iteration space among multiple tasks with `prebody` and `postbody`. It runs those tasks on threads according to a scheduling policy.

Usage:

```julia
@my_threads [schedule] begin
    prebody
end for ...
    ...
end begin
    postbody
end
```


In [4]:
?MyUtils.@my_distributed

```
@my_distributed
```

A distributed memory, parallel for loop of the form:

```julia
@my_distributed begin
    prebody
end [reducer] for var = range
    body
end
```


In [5]:
function mcpi(N)
    rng = Random.default_rng()
    c = 0
    for i in 1:N
        c += ifelse(rand(rng)^2 + rand(rng)^2 ≤ 1, 1, 0)
    end
    4c/N
end

@time mcpi(10^8)
@time mcpi(10^8)
@time mcpi(10^8)

  0.293152 seconds
  0.285791 seconds
  0.280595 seconds


3.14171044

In [6]:
function mcpi_my_threads(N)
    a = Atomic{Int}(0)
    MyUtils.@my_threads begin
        rng = Random.default_rng()
        c = 0
    end for i in 1:N
        c += ifelse(rand(rng)^2 + rand(rng)^2 ≤ 1, 1, 0)
    end begin
        atomic_add!(a, c)
    end
    4a[]/N
end

@time mcpi_my_threads(10^8)
@time mcpi_my_threads(10^8)
@time mcpi_my_threads(10^8)

  0.088797 seconds (59.81 k allocations: 3.130 MiB, 43.97% compilation time)
  0.076956 seconds (80 allocations: 7.969 KiB)
  0.064889 seconds (79 allocations: 7.609 KiB)


3.14163304

In [7]:
rmprocs(procs()[2:end])
addprocs(nthreads())
@show workers()

@everywhere begin
    using MyUtils
    using Random
end

workers() = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [8]:
function mcpi_my_distributed(N)
    c = MyUtils.@my_distributed begin
        rng = Random.default_rng()
    end (+) for i in 1:N
        ifelse(rand(rng)^2 + rand(rng)^2 ≤ 1, 1, 0)
    end
    4c/N
end

@time mcpi_my_distributed(10^8)
@time mcpi_my_distributed(10^8)
@time mcpi_my_distributed(10^8)

  1.183088 seconds (984.30 k allocations: 52.697 MiB, 0.60% gc time, 16.80% compilation time)
  0.057342 seconds (964 allocations: 40.672 KiB)
  0.048361 seconds (975 allocations: 41.391 KiB)


3.14134508

In [9]:
using BenchmarkTools

@btime mcpi(10^8)
@btime mcpi_my_threads(10^8)
@btime mcpi_my_distributed(10^8)

  280.414 ms (0 allocations: 0 bytes)
  48.456 ms (73 allocations: 7.45 KiB)
  48.405 ms (964 allocations: 40.67 KiB)


3.14148876

In [10]:
rmprocs(procs()[2:end])
@show workers();

workers() = [1]


## printf functions

In [11]:
for k in 0:10
    printf("%.$(k)f\n", π)
end

3
3.1
3.14
3.142
3.1416
3.14159
3.141593
3.1415927
3.14159265
3.141592654
3.1415926536


In [12]:
[sprintf("%.$(k)f", π) for k in 0:10]

11-element Vector{String}:
 "3"
 "3.1"
 "3.14"
 "3.142"
 "3.1416"
 "3.14159"
 "3.141593"
 "3.1415927"
 "3.14159265"
 "3.141592654"
 "3.1415926536"

## `RemoveOneFrom(A::AbstractVector, k::Integer) <: AbstractVector`

In [13]:
@doc MyUtils.RemoveOneFrom

```
RemoveOneFrom(A::AbstractVector, k::Integer) <: AbstractVector
```

Equivalent to A[[1:k-1; k+1:end]] without unnecessary memory allocation.

Example

```julia
A = [1, 2, 3, 4, 5]
R = RemoveOneFrom(A, 3)
@show collect(R)
# -> collect(R) = [1, 2, 4, 5]
R[3] = 99
@show R
# -> R = [1, 2, 99, 5]
@show A
# -> A = [1, 2, 3, 99, 5]
```


In [14]:
A = [1, 2, 3, 4, 5]
R = MyUtils.RemoveOneFrom(A, 3)
@show collect(R)
R[3] = 99
@show R
@show A;

collect(R) = [1, 2, 4, 5]
R = [1, 2, 99, 5]
A = [1, 2, 3, 99, 5]


In [15]:
A = collect(1:10^6)
k = 5*10^5
@time A[[1:k-1; k+1:end]]
@time A[[1:k-1; k+1:end]]
@time A[[1:k-1; k+1:end]]
println()
@time @view A[[1:k-1; k+1:end]]
@time @view A[[1:k-1; k+1:end]]
@time @view A[[1:k-1; k+1:end]]
println()
@time MyUtils.RemoveOneFrom(A, k)
@time MyUtils.RemoveOneFrom(A, k)
@time MyUtils.RemoveOneFrom(A, k);

  0.012482 seconds (303 allocations: 15.276 MiB, 70.58% compilation time)
  0.009621 seconds (9 allocations: 15.259 MiB, 61.73% gc time)
  0.003841 seconds (9 allocations: 15.259 MiB)

  0.015765 seconds (23.72 k allocations: 8.860 MiB, 88.98% compilation time)
  0.007379 seconds (8 allocations: 7.630 MiB, 76.72% gc time)
  0.001648 seconds (8 allocations: 7.630 MiB)

  0.000002 seconds (1 allocation: 32 bytes)
  0.000002 seconds (1 allocation: 32 bytes)
  0.000002 seconds (1 allocation: 32 bytes)


## `@savevar` and `@loadvar`

In [16]:
using MyUtils: dir_savevar, @savevar, @loadvar
dir_savevar[] = "tmp"

"tmp"

In [17]:
Random.seed!(4649373)

x = randn(ComplexF64, 4, 3, 2)
y = [randstring(10) for i in 1:50]
@show x y;

x = [0.6139627109171718 + 0.28881204108997827im -0.702730328235708 + 0.8559736028023069im 0.14663020936127483 + 0.7498128557650708im; 0.5956800792827293 - 0.8906783007866128im 0.28211812938862985 - 0.40988464126274915im -0.6470657107865254 + 0.22378412696989403im; 0.08044299916390557 + 0.17968561891711088im 0.31648387135283207 - 0.7446555793850927im 0.30662299351312033 - 0.4195718266129384im; -1.1258631093926221 + 0.21712283581037484im 1.1662912804139092 + 1.2965115750691512im -0.371648943046625 + 1.2600742724940495im;;; 0.3862995751786956 - 1.6414457699539118im 1.3121254331775718 + 0.8139596278083623im -1.2188912807602623 - 0.7951558084918605im; -0.4276250941959636 + 0.30747771286547565im -0.21139027402447005 + 0.23986442754993378im 0.7894121227507885 - 0.9681327534683507im; 0.2782923253259139 - 0.252931480521629im 1.2158580545213769 + 0.9364885079745994im -0.019124302112682224 - 0.35839515408272554im; 0.8614378930030232 + 0.2631196409355939im 0.4751692544034696 - 0.07802820622618017i

In [18]:
@savevar x y

In [19]:
read(joinpath(dir_savevar[], "x.txt"), String)

"[0.6139627109171718 + 0.28881204108997827im -0.702730328235708 + 0.8559736028023069im 0.14663020936127483 + 0.7498128557650708im; 0.5956800792827293 - 0.8906783007866128im 0.28211812938862985 - 0.40988464126274915im -0.6470657107865254 + 0.22378412696989403im; 0.08044299" ⋯ 500 bytes ⋯ "34683507im; 0.2782923253259139 - 0.252931480521629im 1.2158580545213769 + 0.9364885079745994im -0.019124302112682224 - 0.35839515408272554im; 0.8614378930030232 + 0.2631196409355939im 0.4751692544034696 - 0.07802820622618017im -0.9465755208411036 + 0.5357468618775187im]"

In [20]:
read(joinpath(dir_savevar[], "y.txt"), String)

"[\"x8x31b50pw\", \"HLiABYafOZ\", \"Vg0l6os6GW\", \"58q2UKxJgz\", \"zW7dVmMoyN\", \"ByHgqfDdSh\", \"I6CFNHn1Y9\", \"4V503tRrLN\", \"yWfRP1AtuT\", \"zi66QxlzXu\", \"Lsqx36bNd2\", \"BnXTqPuhhS\", \"G6AdqtAnBq\", \"pvUcPsJUg9\", \"bAA6IfCNK6\", \"pjHd1pSeuV\", \"xrYVYmj22v\", \"cn4kxMdqok\", \"eygnQ6VZlp\", \"CL0" ⋯ 159 bytes ⋯ "4G\", \"JrrNnLVa2Q\", \"r4dIvh7LlN\", \"bixGLw38ZO\", \"pm0bW0PKKe\", \"RczpxornAh\", \"bBUQycVK88\", \"9dE5D5bqo3\", \"cvspW8ptuE\", \"wNvhx1vKG9\", \"sXnbFk1lB9\", \"sMSSRGdjXA\", \"XjY5W36Wro\", \"y8oKQ7MSTz\", \"6inL5YLjx6\", \"Wl7SFPPbsZ\", \"xkgp29kitQ\", \"YppeiRZLTU\", \"kuy3KWL7Ho\", \"gn6uqrdXiO\"]"

In [21]:
X, Y = @loadvar x y
@show X Y;

X = [0.6139627109171718 + 0.28881204108997827im -0.702730328235708 + 0.8559736028023069im 0.14663020936127483 + 0.7498128557650708im; 0.5956800792827293 - 0.8906783007866128im 0.28211812938862985 - 0.40988464126274915im -0.6470657107865254 + 0.22378412696989403im; 0.08044299916390557 + 0.17968561891711088im 0.31648387135283207 - 0.7446555793850927im 0.30662299351312033 - 0.4195718266129384im; -1.1258631093926221 + 0.21712283581037484im 1.1662912804139092 + 1.2965115750691512im -0.371648943046625 + 1.2600742724940495im;;; 0.3862995751786956 - 1.6414457699539118im 1.3121254331775718 + 0.8139596278083623im -1.2188912807602623 - 0.7951558084918605im; -0.4276250941959636 + 0.30747771286547565im -0.21139027402447005 + 0.23986442754993378im 0.7894121227507885 - 0.9681327534683507im; 0.2782923253259139 - 0.252931480521629im 1.2158580545213769 + 0.9364885079745994im -0.019124302112682224 - 0.35839515408272554im; 0.8614378930030232 + 0.2631196409355939im 0.4751692544034696 - 0.07802820622618017i

In [22]:
(X, Y) == (x, y)

true

## `@defunpack`

In [23]:
@doc MyUtils.DefUnPack

The `DefUnPack` module only exports the `@defunpack` macro, which defines a macro unpacking properties of an object.

**Simple Intended Usage:**

```julia
julia> using .DefUnPack

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @defunpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(obj) unpacks the fields (:a, :b, :c) of obj.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")
```


In [24]:
@doc @defunpack

```
@defunpack(name::Symbol, expr)
```

defines the macro named `Symbol(:unpack, name)` which unpacks the properties specified by `expr` of an object.

Let `val` be the value of `expr`. Then the list of the unpacking properties is set to

  * `val` if `val` is a tuple of symbols,
  * `fieldnames(val)` if `val` is a type,
  * `propertynames(val)` otherwise.

### Example

```julia
julia> @defunpack _cat_and_dog (:cat, :dog)
@unpack_cat_and_dog (macro with 1 method)

help?> @unpack_cat_and_dog
@unpack_cat_and_dog(obj) unpacks the fields (:cat, :dog) of obj.

julia> @macroexpand @unpack_cat_and_dog x
:((cat, dog) = (x.cat, x.dog))

julia> @unpack_cat_and_dog (dog = "bowwow", mouse = "squeak", cat = "meow")
("meow", "bowwow")

julia> cat, dog
("meow", "bowwow")

julia> struct Foo{A, B, C} a::A; b::B; c::C end

julia> @defunpack all_Foo Foo
@unpackall_Foo (macro with 1 method)

help?> @unpackall_Foo
@unpackall_Foo(obj) unpacks the fields (:a, :b, :c) of obj.

julia> @macroexpand @unpackall_Foo foo
:((a, b, c) = (foo.a, foo.b, foo.c))

julia> @unpackall_Foo Foo(1, 2.0, "three")
(1, 2.0, "three")

julia> a, b, c
(1, 2.0, "three")

julia> baz = (p = "one", q = 2.0, r = 3)

julia> @defunpack all_baz baz
@unpackall_baz (macro with 1 method)

help?> @unpackall_baz
@unpackall_baz(obj) unpacks the fields (:p, :q, :r) of obj.

julia> @macroexpand @unpackall_baz baz
:((p, q, r) = (baz.p, baz.q, baz.r))

julia> @unpackall_baz baz
("one", '2', 3)

julia> p, q, r
("one", '2', 3)
```

### Appendix

Instead of using `@defunpack`, we can also define the unpacking macro directly as follows:

```julia
struct Foo{A, B, C} a::A; b::B; c::C end

"""
`@unpackall_Foo(obj)` unpacks all fields of the object `obj` of type `Foo`.
"""
macro unpackall_Foo(obj)
    names = fieldnames(Foo)
    Expr(:(=),
        Expr(:tuple, names...),
        Expr(:tuple, (:($obj.$name) for name in names)...)
    ) |> esc
end

@macroexpand @unpackall_Foo foo
```

```
:((a, b, c) = (foo.a, foo.b, foo.c))
```


In [25]:
MyUtils.DefUnPack.unpackall_Foo |> print

macro unpackall_Foo(obj)
    names = fieldnames(Foo)
    Expr(:(=),
        Expr(:tuple, names...),
        Expr(:tuple, (:($obj.$name) for name in names)...)
    ) |> esc
end
